In [3]:
import findspark

In [4]:
findspark.init('/home/mukund/spark-2.1.0-bin-hadoop2.7/')

In [5]:
import pyspark

In [6]:
from pyspark.sql import SparkSession

In [7]:
Spark=SparkSession.builder.appName('lreg_sample').getOrCreate()

In [13]:
data=Spark.read.csv('/home/mukund/ml/Ecommerce_Customers.csv',inferSchema=True, header=True)

In [14]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler #used to convert all the features into a feature vector, essential to implement
#machine learning algorithms

In [25]:
data.printSchema()
data.head(2)

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005),
 Row(Email='hduke@hotmail.com', Address='4547 Archer CommonDiazchester, CA 06566-8576', Avatar='DarkGreen', Avg Session Length=31.92627202636016, Time on App=11.109460728682564, Time on Website=37.268958868297744, Length of Membership=2.66403418213262, Yearly Amount Spent=392.2049334443264)]

In [17]:
assembler=VectorAssembler(inputCols=['Avg Session Length','Time on App',
 'Time on Website','Length of Membership'],outputCol='feature')

In [18]:
output=assembler.transform(data)

In [23]:
output.select('feature').head(2) #vectorized version of first two rows

[Row(feature=DenseVector([34.4973, 12.6557, 39.5777, 4.0826])),
 Row(feature=DenseVector([31.9263, 11.1095, 37.269, 2.664]))]

In [27]:
final_data=output.select(['feature','Yearly Amount Spent'])

In [28]:
final_data.show()

+--------------------+-------------------+
|             feature|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [30]:
train_data,test_data=final_data.randomSplit([0.8,0.2])

In [32]:
train_data.describe().show()
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                396|
|   mean|  498.1818010412816|
| stddev|  79.97940638998853|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                104|
|   mean| 503.62524920142164|
| stddev|  76.95586576938473|
|    min| 298.76200786180766|
|    max|  765.5184619388373|
+-------+-------------------+



In [68]:
from pyspark.ml.regression import LinearRegression
lr=LinearRegression(featuresCol='feature',labelCol='Yearly Amount Spent')

In [69]:
model=lr.fit(train_data)

In [70]:
result=model.evaluate(test_data)

In [71]:
result.rootMeanSquaredError

8.909071274827879

In [72]:
result.r2

0.9864675159297834

In [73]:
result.predictions.show() #difference in predictions on test data

+--------------------+-------------------+------------------+
|             feature|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[29.5324289670579...|  408.6403510726275|398.98587443956535|
|[30.7377203726281...|  461.7807421962299|451.28757463402917|
|[31.0662181616375...| 448.93329320767435|462.18035814353493|
|[31.2681042107507...|  423.4705331738239|427.58362328826365|
|[31.5261978982398...|  409.0945261923378| 418.2126470549679|
|[31.5741380228732...|  544.4092721605869| 558.5705595887539|
|[31.5761319713222...|  541.2265839893283| 543.1222301584471|
|[31.6548096756927...|  475.2634237275485|468.45082619708455|
|[31.7366356860502...|  496.9334462555319| 495.0098955276642|
|[31.8124825597242...|  392.8103449837972| 396.8935150789721|
|[31.8164283341993...| 501.12249150365636| 518.2980368472292|
|[31.8530748017465...|   459.285123462352| 461.7339670443464|
|[31.8648325480987...|  439.8912804768137| 449.9916932243916|
|[31.885

In [58]:
#for model deployement, let us create a sample data to test our model for predictions

In [59]:
unlabled1,unlabled2=final_data.randomSplit([0.7,0.2])

In [60]:
unlabled=unlabled1.select('feature')

In [61]:
unlabled.show()

+--------------------+
|             feature|
+--------------------+
|[29.5324289670579...|
|[30.3931845423455...|
|[30.4925366965402...|
|[30.5743636841713...|
|[30.7377203726281...|
|[30.9716756438877...|
|[31.0472221394875...|
|[31.0613251567161...|
|[31.0662181616375...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.2606468698795...|
|[31.2681042107507...|
|[31.3091926408918...|
|[31.3123495994443...|
|[31.3662121671876...|
|[31.3895854806643...|
|[31.4252268808548...|
|[31.4459724827577...|
+--------------------+
only showing top 20 rows



In [74]:
pred=model.transform(unlabled)

In [75]:
pred.show()

+--------------------+------------------+
|             feature|        prediction|
+--------------------+------------------+
|[29.5324289670579...|398.98587443956535|
|[30.3931845423455...|332.09698465001134|
|[30.4925366965402...|288.21998937028934|
|[30.5743636841713...|442.40371858633785|
|[30.7377203726281...|451.28757463402917|
|[30.9716756438877...| 488.1848488903347|
|[31.0472221394875...|388.39477742368103|
|[31.0613251567161...|493.83097574166663|
|[31.0662181616375...|462.18035814353493|
|[31.1239743499119...|508.58740482102485|
|[31.1280900496166...| 564.4424374307421|
|[31.1695067987115...| 416.9502778753954|
|[31.2606468698795...|422.03571616692057|
|[31.2681042107507...|427.58362328826365|
|[31.3091926408918...| 430.0126123265384|
|[31.3123495994443...| 444.9304781999351|
|[31.3662121671876...|427.22652636482144|
|[31.3895854806643...| 409.9346659313294|
|[31.4252268808548...| 534.5181018170576|
|[31.4459724827577...|481.92292328130975|
+--------------------+------------